# Landfill Sentinel — Looking for Methane Plumes

This notebook is designed to spot signs of methane coming from a landfill.  
Methane is invisible, but satellites like **Sentinel-5P** measure how much methane is in the air above the Earth each day. By combining that data with local wind information, it’s possible to “stack up” many days of observations and reveal patterns that point to a source.

The method used here is called **Wind-Rotated Plume Mapping (WRPM)**.  
The basic idea:
1. Collect daily methane maps from the satellite.  
2. Note which way the wind was blowing each day.  
3. Rotate the maps so that the wind always blows north (straight up).  
4. Average them all together.  
5. Look for a “plume” — higher methane values consistently downwind of the landfill.  

This helps confirm whether a landfill is releasing methane.


In [ ]:
import os, io, contextlib
from tempfile import NamedTemporaryFile

import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr

import folium
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import Normalize

import rasterio
from rasterio import warp
from rasterstats import zonal_stats

import openeo
import cdsapi

from shapely.geometry import shape, Point
from tqdm.notebook import tqdm

## Connect to OpenEO

The code below establishes a connection with the Copernicus openEO platform which provides a wide variety of earth observation datasets

- If this does not read as 'Authorised successfully' or 'Authenticated using refresh token', then please ensure that you have completed the setup steps as outlined in section 2.6 of the user guide. 

- If you have followed the steps in section 2.6 correctly and the problem persists, please look at https://dataspace.copernicus.eu/news for any information about service interruptions. 

- If there is no news of service problems you can raise a ticket here: https://helpcenter.dataspace.copernicus.eu/hc/en-gb/requests/new

In [ ]:
connection = openeo.connect(url="openeo.dataspace.copernicus.eu")
connection.authenticate_oidc()

## Define the study area  
The box below sets the landfill’s latitude and longitude and creates a box around it for collecting satellite data.


In [ ]:
user_latitude = -34.529350
user_longitude = -58.618620
buffer_degrees = 1

site_label = f"User site ({user_latitude:.2f},{user_longitude:.2f})"

spatial_extent = {
    "west":  user_longitude - buffer_degrees,
    "south": user_latitude - buffer_degrees,
    "east":  user_longitude + buffer_degrees,
    "north": user_latitude + buffer_degrees,
}


## Choosing date and downloading Sentinel-5P datasets

This downloads the methane concentration data for a particular date. The parameter you need to modify before running the code is:

- temporal_extent = ["2023-01-31", "2023-03-12"] (change this to your chosen date range using "YYYY-MM-DD" format)

Please note that the temporal extent dates <u>MUST BE IDENTICAL</u> because we are only choosing a single date.

In [ ]:
cube = connection.load_collection(
    collection_id="SENTINEL_5P_L2", 
    temporal_extent = ["2024-01-01", "2024-12-31"],
    spatial_extent=spatial_extent,   # <-- from landfill selection cell
    bands=["CH4"],
)

cube = cube.aggregate_temporal_period(period="day", reducer="mean")
cube.download("Sentinel-5P_Landfill_daily.nc", format="NetCDF")
print(f"Downloaded daily Sentinel-5P CH₄ data for {site_label} at {spatial_extent}")


## Add wind information  
Retrieves daily wind speed and direction from ERA5 and matches it to each day’s satellite observation.


In [ ]:
# === Annotate NetCDF daily slices with ERA5 wind ===
# Input: Sentinel-5P_Landfill_daily.nc (t dimension = daily slices), landfill centroid
# Output: winds.csv with [date, wind_speed_mps, wind_from_deg]

in_nc = "Sentinel-5P_Landfill_daily.nc"   # downloaded NetCDF
lat, lon = user_latitude, user_longitude

era5_time = "13:00"                        # UTC near S5P overpass
out_csv = "winds.csv"

# open dataset
ds = xr.open_dataset(in_nc)
da = ds["CH4"]        # CH4 variable
times = pd.to_datetime(ds["t"].values)  # 't' is the time dimension
ds.close()

c = cdsapi.Client()
rows = []

for dt in tqdm(times, desc="Fetching ERA5 winds", unit="day"):
    y, m, d = dt.strftime("%Y"), dt.strftime("%m"), dt.strftime("%d")
    with NamedTemporaryFile(suffix=".nc") as tmp_file:
        # silence cdsapi chatter
        with contextlib.redirect_stdout(io.StringIO()):
            r = c.retrieve(
                "reanalysis-era5-single-levels",
                {
                    "product_type": "reanalysis",
                    "variable": ["10m_u_component_of_wind","10m_v_component_of_wind"],
                    "year": y, "month": m, "day": d,
                    "time": [era5_time],
                    "format": "netcdf",
                    "area": [lat + buffer_degrees, lon - buffer_degrees,
                             lat - buffer_degrees, lon + buffer_degrees],
                },
            )
            r.download(tmp_file.name)
        era = xr.open_dataset(tmp_file.name)
        u10 = era["u10"].sel(latitude=lat, longitude=lon, method="nearest").values.item()
        v10 = era["v10"].sel(latitude=lat, longitude=lon, method="nearest").values.item()
        era.close()

    wind_speed = float(np.hypot(u10, v10))
    wind_from_deg = float((np.degrees(np.arctan2(-u10, -v10)) + 360.0) % 360.0)

    rows.append({
        "date": dt.date().isoformat(),
        "wind_speed_mps": wind_speed,
        "wind_from_deg": wind_from_deg,
    })

pd.DataFrame(rows).sort_values("date").to_csv(out_csv, index=False)
print(f"Wrote {len(rows)} rows to {out_csv}")


## Rotate maps and build a composite  
Turns each daily map so the wind blows north, then stacks them into one combined image where the downwind plume is clearer.


In [ ]:
# === Wind-rotated oversampling to 0.01° (downwind -> NORTH) ===
# Inputs:
#   - Sentinel-5P_Landfill_daily.nc    (var=CH4, time dim = 't', coords x/y)
#   - winds.csv  (columns: date, wind_speed_mps, wind_from_deg)
# Output:
#   - wrpm_oversampled_0p01deg.tif  (lat/lon EPSG:4326, 0.01° grid)

import os
import numpy as np
import pandas as pd
import xarray as xr
import rasterio
from rasterio.transform import from_origin

# ---- params ----
in_nc    = "Sentinel-5P_Landfill_daily.nc"
winds_csv = "winds.csv"
src_lon, src_lat = user_longitude, user_latitude
box_deg = 0.5     # ~1° x 1° box centered on landfill (paper scale)
step    = 0.01    # oversampling grid resolution in degrees (paper)
out_tif = "wrpm_oversampled_0p01deg.tif"
# ----------------

west, east  = src_lon - box_deg, src_lon + box_deg
south, north = src_lat - box_deg, src_lat + box_deg
nx = int(np.round((east - west) / step))
ny = int(np.round((north - south) / step))
dst_transform = from_origin(west, north, step, step)
dst_crs = rasterio.crs.CRS.from_epsg(4326)

# open dataset
ds = xr.open_dataset(in_nc)
da = ds["CH4"]
lat = da.coords["y"]
lon = da.coords["x"]
times = pd.to_datetime(ds["t"].values)

# native pixel half-sizes in degrees (approx footprint)
dy = float(abs(lat[1] - lat[0])) if lat.size > 1 else 0.1
dx = float(abs(lon[1] - lon[0])) if lon.size > 1 else 0.1
hy = 0.5 * dy
hx = 0.5 * dx

# load winds
winds = pd.read_csv(winds_csv, parse_dates=["date"])
winds["date"] = winds["date"].dt.strftime("%Y-%m-%d")

# accumulators
sum_grid   = np.zeros((ny, nx), dtype=np.float64)
count_grid = np.zeros((ny, nx), dtype=np.float64)

def rotate_about_source(lon_arr, lat_arr, src_lon, src_lat, angle_deg):
    """
    Rotate small-area coordinates around (src_lon, src_lat) so that 'angle_deg' aligns to NORTH.
    Positive angle rotates counter-clockwise.
    """
    km_per_deg_lat = 111.32
    km_per_deg_lon = 111.32 * np.cos(np.deg2rad(src_lat))
    x = (lon_arr - src_lon) * km_per_deg_lon
    y = (lat_arr - src_lat) * km_per_deg_lat
    theta = np.deg2rad(angle_deg)
    ct, st = np.cos(theta), np.sin(theta)
    xr =  x * ct - y * st
    yr =  x * st + y * ct
    lon_r = xr / km_per_deg_lon + src_lon
    lat_r = yr / km_per_deg_lat + src_lat
    return lon_r, lat_r

# loop days
for t in times:
    date = t.date().isoformat()
    # extract slice (as array) and subset to speed up
    s = da.sel(t=t)
    # ensure we slice with correct y ordering
    if float(lat[0]) > float(lat[-1]):
        s = s.sel({lat.name: slice(north, south), lon.name: slice(west, east)})
    else:
        s = s.sel({lat.name: slice(south, north), lon.name: slice(west, east)})
    if s.size == 0:
        continue

    LON, LAT = np.meshgrid(s[lon.name].values, s[lat.name].values)
    ARR = np.array(s.values, dtype=np.float32)

    # wind for this date
    wrow = winds[winds["date"] == date]
    if wrow.empty:
        continue
    wind_from = float(wrow.iloc[0]["wind_from_deg"])
    downwind  = (wind_from + 180.0) % 360.0

    # rotate so downwind points NORTH (paper)
    LONr, LATr = rotate_about_source(LON, LAT, src_lon, src_lat, angle_deg=downwind)

    # only keep pixels landing inside the 0.01° canvas; must be finite
    inside = (LONr >= west) & (LONr <= east) & (LATr >= south) & (LATr <= north) & np.isfinite(ARR)
    if not np.any(inside):
        continue
    lon_c = LONr[inside]; lat_c = LATr[inside]; vals = ARR[inside].astype(np.float64)

    # oversample with rectangular footprint (uniform within dx x dy)
    # half-footprint in oversampling index units
    hx_idx = max(1, int(np.ceil(hx / step)))
    hy_idx = max(1, int(np.ceil(hy / step)))

    # map centres to grid indices
    j0 = np.floor((lon_c - west) / step).astype(int)      # 0..nx-1 (east)
    i0 = np.floor((north - lat_c) / step).astype(int)     # 0..ny-1 (southward)

    for ii, jj, v in zip(i0, j0, vals):
        i_min = max(0, ii - hy_idx); i_max = min(ny - 1, ii + hy_idx)
        j_min = max(0, jj - hx_idx); j_max = min(nx - 1, jj + hx_idx)
        sum_grid[i_min:i_max+1, j_min:j_max+1]   += v
        count_grid[i_min:i_max+1, j_min:j_max+1] += 1.0

# mean composite
with np.errstate(invalid="ignore", divide="ignore"):
    comp = (sum_grid / count_grid).astype(np.float32)

profile = {
    "driver": "GTiff",
    "height": ny, "width": nx, "count": 1, "dtype": "float32",
    "crs": dst_crs, "transform": dst_transform,
    "compress": "LZW", "tiled": True, "nodata": np.nan,
}
with rasterio.open(out_tif, "w", **profile) as dst:
    dst.write(comp, 1)
    dst.update_tags(method="WRPM", grid="0.01deg", rotate="downwind→north", oversample="rectangular footprint",
                    bbox=str((west, south, east, north)), step_deg=step)

print(f"Wrote {out_tif}  ({ny} x {nx} at {step}°)")


## Visualise the rotated composite  
Displays the combined image so that any methane plume aligned with the wind can be seen.

In [ ]:
import rasterio, numpy as np, matplotlib.pyplot as plt

p = "wrpm_oversampled_0p01deg.tif"
with rasterio.open(p) as src:
    arr = src.read(1)
    transform = src.transform
    west, north = transform.c, transform.f
    step = transform.a
    east  = west + step*src.width
    south = north - step*src.height

print("Shape:", arr.shape, "| pixel step (deg):", step)
print("Extent (W,S,E,N):", (west, south, east, north))

plt.figure(figsize=(6,6))
plt.imshow(arr, origin="upper", extent=[west, east, south, north])
plt.colorbar(label="CH₄ enhancement (units of input)")
plt.title("WRPM composite (downwind = north)")
plt.xlabel("Longitude"); plt.ylabel("Latitude")
plt.show()


## Build an unrotated composite (comparison)  
Stacks the daily maps without rotation to show why wind alignment is important.

In [ ]:
# === Oversampling to 0.01° WITHOUT rotation (comparison) ===
# Inputs:
#   - Sentinel-5P_Landfill_daily.nc    (var=CH4, time dim='t', coords x/y)
# Output:
#   - oversampled_unrotated_0p01deg.tif  (EPSG:4326, 0.01° grid)

import numpy as np
import pandas as pd
import xarray as xr
import rasterio
from rasterio.transform import from_origin

# ---- params (match your rotated run) ----
in_nc   = "Sentinel-5P_Landfill_daily.nc"
src_lon, src_lat = user_longitude, user_latitude
box_deg = 0.5      # ~1° x 1° box
step    = 0.01     # oversampling grid resolution in degrees
out_tif = "oversampled_unrotated_0p01deg.tif"
# ----------------------------------------

west, east  = src_lon - box_deg, src_lon + box_deg
south, north = src_lat - box_deg, src_lat + box_deg
nx = int(np.round((east - west) / step))
ny = int(np.round((north - south) / step))
dst_transform = from_origin(west, north, step, step)
dst_crs = rasterio.crs.CRS.from_epsg(4326)

# open dataset
ds = xr.open_dataset(in_nc)
da = ds["CH4"]
lat = da.coords["y"]; lon = da.coords["x"]
times = pd.to_datetime(ds["t"].values)

# native pixel half-sizes in degrees (approx footprint)
dy = float(abs(lat[1] - lat[0])) if lat.size > 1 else 0.1
dx = float(abs(lon[1] - lon[0])) if lon.size > 1 else 0.1
hy = 0.5 * dy
hx = 0.5 * dx
hy_idx = max(1, int(np.ceil(hy / step)))
hx_idx = max(1, int(np.ceil(hx / step)))

# accumulators
sum_grid   = np.zeros((ny, nx), dtype=np.float64)
count_grid = np.zeros((ny, nx), dtype=np.float64)

# loop days: subset → oversample (footprint block) → accumulate
for t in times:
    s = da.sel(t=t)
    # subset to analysis box (respect y ordering)
    if float(lat[0]) > float(lat[-1]):
        s = s.sel({lat.name: slice(north, south), lon.name: slice(west, east)})
    else:
        s = s.sel({lat.name: slice(south, north), lon.name: slice(west, east)})
    if s.size == 0:
        continue

    LON, LAT = np.meshgrid(s[lon.name].values, s[lat.name].values)
    ARR = np.array(s.values, dtype=np.float32)

    # take all finite pixels; footprint clipping will limit to canvas
    valid = np.isfinite(ARR)
    if not np.any(valid):
        continue

    lon_c = LON[valid]; lat_c = LAT[valid]; vals = ARR[valid].astype(np.float64)

    # centre indices on the 0.01° grid (no rotation)
    j0 = np.floor((lon_c - west) / step).astype(int)   # 0..nx-1 (east)
    i0 = np.floor((north - lat_c) / step).astype(int)  # 0..ny-1 (southward)

    # rectangular footprint "splat"
    for ii, jj, v in zip(i0, j0, vals):
        i_min = max(0, ii - hy_idx); i_max = min(ny - 1, ii + hy_idx)
        j_min = max(0, jj - hx_idx); j_max = min(nx - 1, jj + hx_idx)
        if i_min > i_max or j_min > j_max:
            continue
        sum_grid[i_min:i_max+1, j_min:j_max+1]   += v
        count_grid[i_min:i_max+1, j_min:j_max+1] += 1.0

# mean composite
with np.errstate(invalid="ignore", divide="ignore"):
    comp = (sum_grid / count_grid).astype(np.float32)

profile = {
    "driver": "GTiff",
    "height": ny, "width": nx, "count": 1, "dtype": "float32",
    "crs": dst_crs, "transform": dst_transform,
    "compress": "LZW", "tiled": True, "nodata": np.nan,
}
with rasterio.open(out_tif, "w", **profile) as dst:
    dst.write(comp, 1)
    dst.update_tags(method="oversample_no_rotation", grid="0.01deg",
                    bbox=str((west, south, east, north)), step_deg=step)

print(f"Wrote {out_tif}  ({ny} x {nx} at {step}°)")


## Check the rotation worked  
Verifies that after rotation the wind really does line up north, ensuring the method is correct.

In [ ]:
# --- WRPM rotation sanity check (should report ~0°) ---

# use local conversion factors at the landfill
km_per_deg_lat = 111.32
km_per_deg_lon = 111.32 * np.cos(np.deg2rad(src_lat))

# build a unit downwind vector (east = sin α, north = cos α)
alpha = np.deg2rad(downwind)
x_loc = np.sin(alpha)   # east component
y_loc = np.cos(alpha)   # north component

# step 1 km along downwind, convert to lon/lat
eps_km = 1.0
dlon = (eps_km * x_loc) / km_per_deg_lon
dlat = (eps_km * y_loc) / km_per_deg_lat
test_lon = src_lon + dlon
test_lat = src_lat + dlat

# rotate coords so that downwind → north
tlon, tlat = rotate_about_source(test_lon, test_lat, src_lon, src_lat, angle_deg=downwind)

# compute bearing after rotation (0° = north)
dx = (tlon - src_lon) * km_per_deg_lon   # east offset in km
dy = (tlat - src_lat) * km_per_deg_lat   # north offset in km
bearing_after = (np.degrees(np.arctan2(dx, dy)) + 360) % 360

print(f"[WRPM check] Downwind={downwind:.1f}° → after rotation ≈ {bearing_after:.1f}° (expect ~0°)")


## Compare downwind and upwind  
Checks whether methane levels are consistently higher downwind than upwind, which would indicate a plume.


In [ ]:
# === Upwind–Downwind Attribution Test (self-contained) ===
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt

# ---- inputs ----
in_nc = "Sentinel-5P_Landfill_daily.nc"   # daily NetCDF (var=CH4, time='t', coords x/y)
winds_csv = "winds.csv"                    # has columns: date, wind_from_deg, wind_speed_mps
src_lon, src_lat = user_longitude, user_latitude
box_deg = 0.6                              # subset around source for speed
# downwind/upwind boxes in rotated (x=east km, y=north km) frame:
along_km = 20.0
cross_km = 10.0
min_pixels_per_day = 1
n_perm = 1000
# ----------------

# helper: rotate lon/lat about (lon0,lat0) by +angle_deg (CCW), small-area approx
def rotate_about_source(lon_arr, lat_arr, lon0, lat0, angle_deg):
    km_per_deg_lat = 111.32
    km_per_deg_lon = 111.32 * np.cos(np.deg2rad(lat0))
    x = (lon_arr - lon0) * km_per_deg_lon  # east km
    y = (lat_arr - lat0) * km_per_deg_lat  # north km
    th = np.deg2rad(angle_deg)
    ct, st = np.cos(th), np.sin(th)
    xr =  x * ct - y * st
    yr =  x * st + y * ct
    lon_r = xr / km_per_deg_lon + lon0
    lat_r = yr / km_per_deg_lat + lat0
    return lon_r, lat_r

# load data & winds
ds = xr.open_dataset(in_nc)
da = ds["CH4"]
lat = da.coords["y"]; lon = da.coords["x"]
times = pd.to_datetime(ds["t"].values)

wdf = pd.read_csv(winds_csv, parse_dates=["date"])
wdf["date"] = wdf["date"].dt.strftime("%Y-%m-%d")
wind_map = dict(zip(wdf["date"], wdf["wind_from_deg"]))

contrasts = []
for t in times:
    date = t.date().isoformat()
    if date not in wind_map:
        continue
    wind_from = float(wind_map[date])
    downwind  = (wind_from + 180.0) % 360.0

    # subset around source (respect y ordering)
    s = da.sel(t=t)
    if float(lat[0]) > float(lat[-1]):
        s = s.sel({lat.name: slice(src_lat+box_deg, src_lat-box_deg),
                   lon.name: slice(src_lon-box_deg, src_lon+box_deg)})
    else:
        s = s.sel({lat.name: slice(src_lat-box_deg, src_lat+box_deg),
                   lon.name: slice(src_lon-box_deg, src_lon+box_deg)})
    if s.size == 0:
        continue

    # arrays
    LON, LAT = np.meshgrid(s[lon.name].values, s[lat.name].values)
    ARR = np.array(s.values, dtype=np.float32)

    # rotate so downwind points NORTH (use +downwind)
    LONr, LATr = rotate_about_source(LON, LAT, src_lon, src_lat, angle_deg=downwind)

    # local-plane coords (km)
    km_per_deg_lat = 111.32
    km_per_deg_lon = 111.32 * np.cos(np.deg2rad(src_lat))
    Xkm = (LONr - src_lon) * km_per_deg_lon
    Ykm = (LATr - src_lat) * km_per_deg_lat

    valid = np.isfinite(ARR)
    dw_mask = valid & (Ykm >= 0.0) & (Ykm <= along_km) & (np.abs(Xkm) <= cross_km)
    uw_mask = valid & (Ykm <= 0.0) & (Ykm >= -along_km) & (np.abs(Xkm) <= cross_km)

    if dw_mask.sum() < min_pixels_per_day or uw_mask.sum() < min_pixels_per_day:
        continue

    contrasts.append(float(np.nanmean(ARR[dw_mask]) - np.nanmean(ARR[uw_mask])))

ds.close()

contrasts = np.array(contrasts, dtype=float)
if contrasts.size == 0:
    raise RuntimeError("No valid days satisfied the masks; try increasing box_deg or lowering min_pixels_per_day.")

effect = float(np.nanmean(contrasts))
stderr = float(np.nanstd(contrasts, ddof=1) / np.sqrt(len(contrasts)))
z = effect / (stderr + 1e-12)

# permutation (sign-flip null)
rng = np.random.default_rng(42)
perm_effects = np.array([np.mean(contrasts * rng.choice([-1,1], size=len(contrasts))) for _ in range(n_perm)])
p = float((np.sum(np.abs(perm_effects) >= abs(effect)) + 1) / (n_perm + 1))

print(f"Days used: {len(contrasts)}")
print(f"Downwind–Upwind mean diff = {effect:.2f}, SE={stderr:.2f}, z={z:.2f}, p≈{p:.3f}")

# plots
plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
plt.hist(contrasts, bins=20, edgecolor="k")
plt.axvline(0, color="k", linestyle="--")
plt.axvline(effect, color="red", linestyle="-", label=f"Mean={effect:.1f}")
plt.title("Histogram of daily contrasts (downwind–upwind)")
plt.xlabel("CH4 difference"); plt.ylabel("Days"); plt.legend()

plt.subplot(1,2,2)
plt.boxplot(contrasts, vert=True, labels=["Contrasts"])
plt.axhline(0, color="k", linestyle="--")
plt.title("Boxplot of daily contrasts")
plt.ylabel("CH4 difference")
plt.tight_layout()
plt.show()
